In [1]:
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")

In [2]:
#basic 14 day rsi strategy
#buy when RSI < 30, sell when RSI > 70

import vectorbt as vbt
import yfinance as yf

# === 1. Load Data ===
symbol = 'SPY'  # You can change this to any stock symbol you want
price = yf.Ticker(symbol).history(period='max', interval='1d')['Close']
#use freq('D') to ensure daily frequency
price = price.asfreq('D').fillna(method='ffill')

# === 2. Compute RSI ===
rsi = vbt.RSI.run(price, window=14)

# === 3. Define Entry and Exit Conditions ===
entries = rsi.rsi < 40
exits = rsi.rsi > 60

# === 4. Simulate Portfolio ===
portfolio = vbt.Portfolio.from_signals(price, entries, exits, direction='longonly')


# === 5. Analyze Performance ===
trades = portfolio.trades.records_readable
pnl = trades['PnL']

# Compute the rolling win rate (over last 10 trades)
rolling_win_rate = pnl.rolling(window=100).apply(lambda x: (x > 0).mean(), raw=False)

# Use the Exit Timestamp as x-axis for time context
exit_dates = trades['Exit Timestamp'].iloc[rolling_win_rate.index]

# Plotting
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=exit_dates, y=rolling_win_rate, mode='lines', name='Rolling Win Rate'))

# Horizontal lines
fig.add_hline(y=0.5, line_dash='dash', line_color='red',
              annotation_text='50% Win Rate', annotation_position='bottom right')
fig.add_hline(y=0.75, line_dash='dash', line_color='green',
              annotation_text='75% Win Rate', annotation_position='bottom right')
fig.add_hline(y=0.3, line_dash='dash', line_color='blue',
              annotation_text='30% Win Rate', annotation_position='bottom right')
fig.add_hline(y=.9, line_dash='dash', line_color='black'
                , annotation_text='90% Win Rate', annotation_position='bottom right')

fig.update_layout(title='Rolling Win Rate of RSI Strategy',
                  xaxis_title='Trade Exit Date', yaxis_title='Win Rate')
fig.show()

C:\Users\kevin\AppData\Local\Temp\ipykernel_33904\2339176456.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price = price.asfreq('D').fillna(method='ffill')


In [4]:
import yfinance as yf
import vectorbt as vbt

# Load SPY historical price data
price = yf.Ticker('SPY').history(period='max', interval='1d')['Close']
price = price.asfreq('D').fillna(method='ffill')

# Calculate 14-day RSI
rsi = vbt.RSI.run(price, window=14)

# Count how many times RSI dropped below 30
count_under_30 = (rsi.rsi < 30).sum()
print(f"RSI < 30 occurred {count_under_30} times.")

RSI < 30 occurred 1100 times.


C:\Users\kevin\AppData\Local\Temp\ipykernel_33904\3428502846.py:6: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

